##Lesson 2 - developing query with database instead of downloading eveything

In [1]:
from astroquery.gaia import Gaia 

Created TAP+ (v1.2.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443
Created TAP+ (v1.2.1) - Connection:
	Host: geadata.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443


In [7]:
query = """SELECT
COUNT(source_id)
FROM gaiadr2.gaia_source
WHERE 1 = CONTAINS( 
     POINT(ra,dec),
     CIRCLE(266.4,-29.0,0.08))"""

In [9]:
job_ex1 = Gaia.launch_job(query)
result_ex1=job.get_results()
result_ex1

count
int64
1658


In [10]:
#next we will learn to rotate the coordinates into the GD-1 frame of motion.
#so we need to learn coordinates and units

In [11]:
import astropy.units as u 

In [12]:
dir(u) #to see available units

['A',
 'AA',
 'AB',
 'ABflux',
 'ABmag',
 'AU',
 'Angstrom',
 'B',
 'Ba',
 'Barye',
 'Bi',
 'Biot',
 'Bol',
 'Bq',
 'C',
 'Celsius',
 'Ci',
 'CompositeUnit',
 'D',
 'Da',
 'Dalton',
 'Debye',
 'Decibel',
 'DecibelUnit',
 'Dex',
 'DexUnit',
 'EA',
 'EAU',
 'EB',
 'EBa',
 'EC',
 'ED',
 'EF',
 'EG',
 'EGal',
 'EH',
 'EHz',
 'EJ',
 'EJy',
 'EK',
 'EL',
 'EN',
 'EOhm',
 'EP',
 'EPa',
 'ER',
 'ERy',
 'ES',
 'ESt',
 'ET',
 'EV',
 'EW',
 'EWb',
 'Ea',
 'Eadu',
 'Earcmin',
 'Earcsec',
 'Eau',
 'Eb',
 'Ebarn',
 'Ebeam',
 'Ebin',
 'Ebit',
 'Ebyte',
 'Ecd',
 'Echan',
 'Ecount',
 'Ect',
 'Ed',
 'Edeg',
 'Edyn',
 'EeV',
 'Eerg',
 'Eg',
 'Eh',
 'EiB',
 'Eib',
 'Eibit',
 'Eibyte',
 'Ek',
 'El',
 'Elm',
 'Elx',
 'Elyr',
 'Em',
 'Emag',
 'Emin',
 'Emol',
 'Eohm',
 'Epc',
 'Eph',
 'Ephoton',
 'Epix',
 'Epixel',
 'Erad',
 'Es',
 'Esr',
 'Eu',
 'Evox',
 'Evoxel',
 'Eyr',
 'F',
 'Farad',
 'Fr',
 'Franklin',
 'FunctionQuantity',
 'FunctionUnitBase',
 'G',
 'GA',
 'GAU',
 'GB',
 'GBa',
 'GC',
 'GD',
 'GF',
 '

In [13]:
ra = 30*u.deg
type(ra)

astropy.units.quantity.Quantity

In [14]:
print(ra)  #tells us it is degrees!

30.0 deg


In [15]:
ra   #will render!

<Quantity 30. deg>

In [16]:
phi1_min = -55
phi1_max = -45
phi2_min = -8
phi2_max = 4

In [17]:
#define lower left go around clockwise
phi1_rect=[phi1_min,phi1_max,phi1_max,phi1_min]*u.deg
phi2_rect=[phi2_min,phi2_min,phi2_max,phi2_max]*u.deg

In [18]:
phi1_rect

<Quantity [-55., -45., -45., -55.] deg>

In [22]:
import gala.coordinates as gc

In [24]:
corners = gc.GD1Koposov10(phi1=phi1_rect,phi2=phi2_rect)

In [25]:
type(corners)

gala.coordinates.gd1.GD1Koposov10

In [26]:
corners

<GD1Koposov10 Coordinate: (phi1, phi2) in deg
    [(-55., -8.), (-45., -8.), (-45.,  4.), (-55.,  4.)]>

In [27]:
import astropy.coordinates as coord

In [28]:
corners_icrs=corners.transform_to(coord.ICRS)

In [29]:
corners_icrs


<ICRS Coordinate: (ra, dec) in deg
    [(146.27533314, 19.26190982), (152.81671045, 27.13611254),
     (141.60264825, 34.3048303 ), (135.42163944, 25.87738723)]>

In [78]:
###
query = """SELECT
COUNT(source_id)
FROM gaiadr2.gaia_source
WHERE 
    parallax<1
    AND bp_rp BETWEEN -0.75 AND 2.0
    AND 1 = CONTAINS( 
     POINT(ra,dec),
     POLYGON()"""

In [79]:
for point in corners_icrs:
    print(point.ra,point.dec)

146d16m31.1993s 19d15m42.8754s
152d49m00.1576s 27d08m10.0051s
141d36m09.5337s 34d18m17.3891s
135d25m17.902s 25d52m38.594s


In [80]:
for point in corners_icrs:
    print(point.ra.value,point.dec.value)

146.27533313607782 19.261909820533692
152.81671044675923 27.136112541397996
141.60264825107333 34.304830296257144
135.42163944306296 25.877387227672134


In [81]:
point_base = "{point.ra.value}, {point.dec.value}"
#instead of doing for loop use list comprehension
t = [point_base.format(point=point)
     for point in corners_icrs]

In [82]:
t

['146.27533313607782, 19.261909820533692',
 '152.81671044675923, 27.136112541397996',
 '141.60264825107333, 34.304830296257144',
 '135.42163944306296, 25.877387227672134']

In [83]:
#now joint these into a single string
point_list = ', '.join(t)
point_list

'146.27533313607782, 19.261909820533692, 152.81671044675923, 27.136112541397996, 141.60264825107333, 34.304830296257144, 135.42163944306296, 25.877387227672134'

In [84]:
query_base = """SELECT
{columns}
FROM gaiadr2.gaia_source
WHERE parallax<1
     AND bp_rp BETWEEN -0.75 AND 2.0
     AND 1 = CONTAINS( 
     POINT(ra,dec),
     POLYGON({point_list}))"""

In [85]:
columns = 'source_id, ra, dec, pmra, pmdec, parallax, parallax_error, radial_velocity'

In [86]:
query = query_base.format(columns=columns,point_list=point_list)

In [87]:
print(query)

SELECT
source_id, ra, dec, pmra, pmdec, parallax, parallax_error, radial_velocity
FROM gaiadr2.gaia_source
WHERE parallax<1
     AND bp_rp BETWEEN -0.75 AND 2.0
     AND 1 = CONTAINS( 
     POINT(ra,dec),
     POLYGON(146.27533313607782, 19.261909820533692, 152.81671044675923, 27.136112541397996, 141.60264825107333, 34.304830296257144, 135.42163944306296, 25.877387227672134))


In [88]:
job=Gaia.launch_job_async(query)
print(job)

INFO: Query finished. [astroquery.utils.tap.core]
<Table length=140340>
      name       dtype    unit                              description                             n_bad 
--------------- ------- -------- ------------------------------------------------------------------ ------
      source_id   int64          Unique source identifier (unique within a particular Data Release)      0
             ra float64      deg                                                    Right ascension      0
            dec float64      deg                                                        Declination      0
           pmra float64 mas / yr                         Proper motion in right ascension direction      0
          pmdec float64 mas / yr                             Proper motion in declination direction      0
       parallax float64      mas                                                           Parallax      0
 parallax_error float64      mas                                        

In [89]:
results = job.get_results()
len(results)

140340

In [90]:
type(results)

astropy.table.table.Table

In [91]:
filename = 'gd1_results.fits'
results.write(filename, overwrite=True)


In [92]:
ls -l 


total 17976
-rw-r--r--  1 gizis  staff    25726 Nov 17 13:05 01Lesson_gaia.ipynb
-rw-r--r--  1 gizis  staff    36458 Nov 17 15:15 2Lesson_units.ipynb
-rw-r--r--  1 gizis  staff   137067 Nov 17 15:13 3Lesson_plots.ipynb
-rw-r--r--  1 gizis  staff  8994240 Nov 17 15:17 gd1_results.fits
-rw-r--r--@ 1 gizis  staff     3228 Nov 16 12:57 test_setup.ipynb


In [93]:
!ls -lh gd1_results.fits

-rw-r--r--  1 gizis  staff   8.6M Nov 17 15:17 gd1_results.fits


In [94]:
#end of lesson 2.  
#was:
# more ADQL
#astropy units
#Gala coordinates
#Astropy table and FITS files